In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
file_path = '/content/Video_Game_Information.csv'
video_game_data = pd.read_csv(file_path)

In [3]:
# Encode categorical variables
label_encoders = {}
for column in ['Title', 'Genre', 'Platform']:
    le = LabelEncoder()
    video_game_data[column] = le.fit_transform(video_game_data[column])
    label_encoders[column] = le

In [4]:
# Normalize numerical variables
scaler = MinMaxScaler()
video_game_data[['ReleaseYear', 'NumPlayers']] = scaler.fit_transform(video_game_data[['ReleaseYear', 'NumPlayers']])

In [5]:
# Prepare features and labels
X = video_game_data[['Title', 'Genre', 'Platform', 'ReleaseYear', 'NumPlayers']].values
y = video_game_data['AvgRating'].values

In [6]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [8]:
# Define the model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
# Reshape the data to fit LSTM input
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [11]:
# Train the model
history = model.fit(X_train_reshaped, y_train, batch_size=1, epochs=10, validation_data=(X_test_reshaped, y_test))

Epoch 1/10
97/97 [==============================] - 6s 21ms/step - loss: 11.4523 - val_loss: 0.7794
Epoch 2/10
97/97 [==============================] - 1s 9ms/step - loss: 1.0045 - val_loss: 1.0779
Epoch 3/10
97/97 [==============================] - 1s 9ms/step - loss: 1.3698 - val_loss: 0.4901
Epoch 4/10
97/97 [==============================] - 1s 7ms/step - loss: 0.9436 - val_loss: 0.6518
Epoch 5/10
97/97 [==============================] - 1s 6ms/step - loss: 0.8852 - val_loss: 0.4615
Epoch 6/10
97/97 [==============================] - 1s 6ms/step - loss: 1.0108 - val_loss: 0.4268
Epoch 7/10
97/97 [==============================] - 1s 6ms/step - loss: 0.8322 - val_loss: 0.3451
Epoch 8/10
97/97 [==============================] - 1s 8ms/step - loss: 0.8383 - val_loss: 0.3316
Epoch 9/10
97/97 [==============================] - 1s 7ms/step - loss: 0.7098 - val_loss: 0.2953
Epoch 10/10
97/97 [==============================] - 1s 6ms/step - loss: 1.2202 - val_loss: 0.2959


In [12]:
# Evaluate the model
loss = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {loss}')

1/1 [==============================] - 1s 991ms/step - loss: 0.2959
Test Loss: 0.2959263026714325


In [13]:
# Make predictions
predictions = model.predict(X_test_reshaped)

1/1 [==============================] - 1s 865ms/step


In [14]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

# Define a custom accuracy metric
def custom_accuracy(y_true, y_pred, threshold=0.5):
    correct_predictions = np.abs(y_true - y_pred) <= threshold
    accuracy = np.mean(correct_predictions)
    return accuracy

# Calculate custom accuracy
accuracy = custom_accuracy(y_test, predictions)
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error: 0.438441146850586
Custom Accuracy: 61.44%
